In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('incidents.csv')
wa=df[df['state'] == 'Washington']
wa

,incident_id,date,state,city_or_county,n_killed,n_injured,latitude,longitude
45,483817,2013-03-31,Washington,Auburn,3,2,47.3149,-122.224
54,484333,2013-04-21,Washington,Federal Way,5,0,47.3030,-122.310
153,491281,2013-07-29,Washington,Granger,0,4,46.3421,-120.187
193,480071,2013-09-15,Washington,Yakima,0,6,46.5854,-120.538
323,92345,2014-01-01,Washington,Georgetown,0,1,47.5498,-122.318
...,...,...,...,...,...,...,...,...
239176,1079791,2018-03-28,Washington,Seattle,0,0,47.6138,-122.332
239399,1080903,2018-03-29,Washington,Pasco,0,1,NaN,NaN
239401,1081610,2018-03-29,Washington,Walla Walla,0,0,46.0737,-118.312
239402,1081612,2018-03-29,Washington,Yakima,0,1,46.6173,-120.512


In [3]:
import pandas as pd
import datetime
from pandarallel import pandarallel
from nltk.sentiment.vader import SentimentIntensityAnalyzer
pandarallel.initialize(nb_workers=32)
import twint
import time
import random
import nltk
import nest_asyncio
nest_asyncio.apply()
# __import__('IPython').embed()

nltk.download('vader_lexicon')

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Python 3.6.10 (default, Apr 19 2020, 18:31:46) 
Type 'copyright', 'credits' or 'license' for more information
IPython 7.16.1 -- An enhanced Interactive Python. Type '?' for help.

In [1]: quit()



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/yeraf/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
radius = 2
over = 14
def get_tweet_loc(params):
    for _ in range(10):
        try:
            c = twint.Config()
            c.Search = params['keyword']
            c.Limit = 100
            c.Geo = f"{str(params['lat'])},{str(params['lng'])},{str(params['distance'])}{params['unit']}"
            c.Pandas = True
            c.Since = params['since']
            c.Until = params['until']
            twint.run.Search(c)
            sid = SentimentIntensityAnalyzer()
            stat = {"pos": 0, "neg": 0, "neu": 0, "comp": 0, "scrape": True}
            print(twint.storage.panda.Tweets_df.shape)
            if twint.storage.panda.Tweets_df.shape[0] > 0:
                tweets = twint.storage.panda.Tweets_df['tweet']
                for tweet in tweets:
                    senvalue = sid.polarity_scores(tweet)
                    stat['pos'] += senvalue["pos"]
                    stat['neg'] += senvalue["neg"]
                    stat['neu'] += senvalue["neu"]
                    stat['comp'] += senvalue["compound"]
                stat['neg'] /= tweets.shape[0]
                stat['pos'] /= tweets.shape[0]
                stat['neu'] /= tweets.shape[0]
                stat['comp'] /= tweets.shape[0] 
            return stat
        except Exception as e:
            time.sleep(random.random() * 300)
            print('error: ', e)
            continue
    return {"pos": 0, "neg": 0, "neu": 0, "comp": 0, "scrape": True}

def get_loca_polar(r):
    response = get_tweet_loc({
            "keyword": "gun",
            "lat": r['latitude'],
            "lng": r['longitude'],
            "distance": radius,
            "unit": "km",
            "since": (datetime.datetime.strptime(r['date'], '%Y-%m-%d') - datetime.timedelta(days=over)).strftime('%Y-%m-%d'),
            "until": (datetime.datetime.strptime(r['date'], '%Y-%m-%d') + datetime.timedelta(days=over)).strftime('%Y-%m-%d')
        })
    print(response)
    if response:
        r['pos'] = response['pos']
        r['neg'] = response['neg']
        r['neu'] = response['neu']
        r['comp'] = response['comp']
        r['scrape'] = response['scrape']
    else:
        r['pos'] = 0
        r['neg'] = 0
        r['neu'] = 0
        r['comp'] = 0
        return r

wa_polar = wa.parallel_apply(get_loca_polar, axis=1)
wa_polar.to_csv('wa_polar_1_14d.csv')

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.[!] No more data! Scraping will stop now.[!] No more data! Scraping will stop now.


[!] No more data! Scraping will stop now.found 0 deleted tweets in this search.[!] No more data! Scraping will stop now.[!] No more data! Scraping will stop now.found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.481122705245085696 2014-06-24 00:12:46 +0700 <welyons14> Guns only intensify every situation.   #theview
[!] No more data! Scraping will stop now.found 0 deleted tweets in this search.

(0, 0)
708114513581584384 2016-03-11 09:17:10 +0700 <ac_anderson> My ski bag is camo &amp; I was carrying it to checkin @ the airport &amp; a guy yells "damn she got a big gun!" Yes, say hello to my little friends.found 0 deleted tweets in this search.



[!] No more data! Scraping will stop now.found 0 deleted tweets in this search.

479455635080499200 2014-06-19

(4, 38){'pos': 0.0865, 'neg': 0.2665, 'neu': 0.647, 'comp': -0.14425000000000002, 'scrape': True}

594401679392530432 2015-05-02 14:22:58 +0700 <JeffMancubEMD> "That's a god damn mini gun!"  http://t.co/iaiDYDPl8K872121858665046017 2017-06-06 23:03:49 +0700 <EchemGerwe> Bigger than the Beatles Bigger than breast implants Bigger than guns, bigger than cigarettes
[!] No more data! Scraping will stop now.
[!] No more data! Scraping will stop now.{'pos': 0.021, 'neg': 0.24775000000000003, 'neu': 0.7315, 'comp': -0.407825, 'scrape': True}


593990431018356737 2015-05-01 11:08:49 +0700 <mrproam> @SenTomCotton you don't know shit..how dare you...I'll debate you on any subject except guns,ammo and killing..i know little about them
found 0 deleted tweets in this search.871859305640632321 2017-06-06 05:40:31 +0700 <Torbiesmom> Lone Wolf? Mental Health Issues? Another example of why white men with guns are the biggest threat to our safety? #OrlandoShooting  https://t.co/m5v14ylKg4(2, 38)
59387216

ERROR:asyncio:Exception in callback BaseSelectorEventLoop._sock_connect_cb(<Future finished result=None>, <socket.socke....117', 57674)>, ('104.244.42.66', 443))
handle: <Handle BaseSelectorEventLoop._sock_connect_cb(<Future finished result=None>, <socket.socke....117', 57674)>, ('104.244.42.66', 443))>
Traceback (most recent call last):
  File "/home/yeraf/.pyenv/versions/3.6.10/lib/python3.6/asyncio/events.py", line 145, in _run
    self._callback(*self._args)
  File "/home/yeraf/.pyenv/versions/3.6.10/lib/python3.6/asyncio/selector_events.py", line 476, in _sock_connect_cb
    fut.set_result(None)
  File "/home/yeraf/.pyenv/versions/3.6.10/lib/python3.6/asyncio/futures.py", line 295, in set_result
    raise InvalidStateError('{}: {!r}'.format(self._state, self))
asyncio.base_futures.InvalidStateError: FINISHED: <Future finished result=None>

ERROR:asyncio:Exception in callback BaseSelectorEventLoop._sock_connect_cb(<Future finished result=None>, <socket.socke....117', 57702)>, ('104.244.42.66', 443))
handle: <Handle BaseSelectorEventLoop._sock_connect_cb(<Future finished result=None>, <socket.socke....117', 57702)>, ('104.244.42.66', 443))>
Traceback (most recent call last):
  File "/home/yeraf/.pyenv/versions/3.6.10/lib/python3.6/asyncio/events.py", line 145, in _run
    self._callback(*self._args)
  File "/home/yeraf/.pyenv/versions/3.6.10/lib/python3.6/asyncio/selector_events.py", line 476, in _sock_connect_cb
    fut.set_result(None)
  File "/home/yeraf/.pyenv/versions/3.6.10/lib/python3.6/asyncio/futures.py", line 295, in set_result
    raise InvalidStateError('{}: {!r}'.format(self._state, self))
asyncio.base_futures.InvalidStateError: FINISHED: <Future finished result=None>

found 0 deleted tweets in this search.

...

641647290613002240 2015-09-09 23:20:09 +0700 <Helen47> @Helen47  Reminds me, it's that time of the year to get to the gun range.
[!] No more data! Scraping will stop now.641374134174257152 2015-09-09 05:14:44 +0700 <Helen47> @notalemming  Well, in my opinion, Liberals SHOULDN'T own guns. : )

found 0 deleted tweets in this search.640967552710017024 2015-09-08 02:19:07 +0700 <Helen47> @RitaWheater @SkotCG   Idiot Liberals who "hate" guns do.

638845438389391361 2015-09-02 05:46:36 +0700 <MichaelFS1969> Since 1993, Americans acquired over 170 million new firearms, firearm homicides have fallen by 53%. More Guns, Less Crime. #GunsSaveLives
636673072221712384 2015-08-27 05:54:23 +0700 <vestal13> "@politico: 'It's got to stop': 15 times Obama pushed for stronger gun control  http://t.co/sYC3yQYnnt / we're proving that it doesn't
(61, 38)
{'pos': 0.036803278688524586, 'neg': 0.22706557377049175, 'neu': 0.7361311475409835, 'comp': -0.4351901639344264, 'scrape': True}
[!] No more data! Scrapin

629016330499330048 2015-08-06 02:49:14 +0700 <SeattleCP> Police: Attacker armed with hatchet, gun opens fire inside Nashville area movie theater: NASHVILLE, Tennessee — A…  http://t.co/tO45U3l5s9
628956400928948225 2015-08-05 22:51:06 +0700 <SeattleCP> No. 2 Senate Republican proposing gun-background-check bill: Backed by the National Rifle Association, the Senate’s…  http://t.co/Sz7e8XQky8
628723121768390656 2015-08-05 07:24:07 +0700 <SeattleCP> Ted Cruz releases video of bacon cooked on gun barrel: The minute-long video was posted by the Independent Journal…  http://t.co/yYnhJcponp
628723120115855360 2015-08-05 07:24:07 +0700 <SeattleCP> Ted Cruz releases video of bacon cooked on gun barrel: The minute-long video was posted by the Independent Journal…  http://t.co/Kfdn0yu9rA
628702753385115649 2015-08-05 06:03:11 +0700 <SeattleCP> Lakebay Marina evacuated after man fires gun, claims to have bombs, deputies say: PIERCE COUNTY, Wash. — Lakebay…  http://t.co/060Wttqnke
62840212217058918

{'pos': 0, 'neg': 0, 'neu': 0, 'comp': 0, 'scrape': True}
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
(0, 0)
{'pos': 0, 'neg': 0, 'neu': 0, 'comp': 0, 'scrape': True}
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
(0, 0)
{'pos': 0, 'neg': 0, 'neu': 0, 'comp': 0, 'scrape': True}
627485724657389569 2015-08-01 21:27:09 +0700 <vestal13> Guns and the Two Americas  http://t.co/hR6Uli6Abh
626326758426411008 2015-07-29 16:41:50 +0700 <ZtheDopest> Doesnt hurt to blow things outta proportion and jump the gun once in a while. Lost 10lbs jumpin conclusions tbh  https://t.co/fT0nGtAE50
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
(0, 0)
{'pos': 0, 'neg': 0, 'neu': 0, 'comp': 0, 'scrape': True}
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
(2, 38)
{'pos': 0.0565, 'neg': 0.096, 'neu': 0.8474999999999999, 'comp': -0.11605, 'scrape': True}
error:  Could not find 

644585130896429056 2015-09-18 01:54:05 +0700 <SeattleCP> Seattle police say gains made in fight against South Seattle gun violence: SEATTLE -- While it is too early to…  http://t.co/rTb6Xx4AjJ
644227774975119360 2015-09-17 02:14:05 +0700 <SeattleCP> South Seattle funeral director mourns tragic toll of gun violence: SEATTLE -- Roger Leger is tired of burying dead…  http://t.co/CL1BBniKSe[!] No more data! Scraping will stop now.

643812694630359040 2015-09-15 22:44:42 +0700 <kiusau> Yes, thieves would find it more difficult to rob, if their victims had guns. This is why the state would take them away. It's a thief!found 0 deleted tweets in this search.

643409283728539649 2015-09-14 20:01:41 +0700 <sleepylemur> @warisboring @alizardx more like "smoke a joint, strike fighter" amirite  No infrared dogfighting or cruise missiles, GPS bombs, or guns?!
643321539102941184 2015-09-14 14:13:01 +0700 <1sfreedom> National Journal “Study” Promoting Gun Control: “F” Stands for Fail  https://t.co/Gzl

636893953913040896 2015-08-27 20:32:06 +0700 <sleepylemur> @BrettHamil death from gun violence is tragic, death from car crashes is tragic (negligence / murderous intent may vary)  Pollution is worse
636759459251904512 2015-08-27 11:37:40 +0700 <josexskates> USED TO TRE FLIP NOW NIGGAS KEEPIN GUNS WIT EM
636706496345038848 2015-08-27 08:07:12 +0700 <SeattleCP> The NRA’s Olympia lobbyist defends ivory handled guns, takes on “huge billionaire” Paul Allen: The National Rifle…  http://t.co/OhQWcXqihI
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
(63, 38)
{'pos': 0.07942857142857142, 'neg': 0.21879365079365085, 'neu': 0.7017777777777777, 'comp': -0.3233063492063491, 'scrape': True}
978796023312138240 2018-03-28 07:48:51 +0700 <KL_Baseball> The "Sheriff"  L. VonGoedert guns them down 1-2-3 in the 7th to close out the Falcons 8th WIN! #YouGottaLoveTheseGuys  https://t.co/j8mFc6wMem
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this sear